<a href="https://colab.research.google.com/github/daeun118/MachineLearning1/blob/main/%EB%A8%B8%EC%8B%A0%EB%9F%AC%EB%8B%9D1_3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

랜덤포레스트로 피쳐 중요도 확인 후 피쳐 삭제->svm사용하여 모델 구축

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
data=np.load('/content/drive/MyDrive/train.npy', allow_pickle='TRUE')
data=data.item()
input_data = data['input']
input_data

array([[[[-2.8890e+00],
         [-7.9550e+00],
         [-1.6170e+00],
         ...,
         [ 1.9430e+00],
         [-3.0920e+00],
         [-1.7090e+00]],

        [[-3.3770e+00],
         [-8.9310e+00],
         [-1.6170e+00],
         ...,
         [ 2.9200e+00],
         [-3.0920e+00],
         [ 2.4400e-01]],

        [[-3.3770e+00],
         [-8.9310e+00],
         [-1.5300e-01],
         ...,
         [ 3.4080e+00],
         [-3.0920e+00],
         [ 2.1970e+00]],

        ...,

        [[ 5.9000e+00],
         [ 3.7640e+00],
         [-8.9420e+00],
         ...,
         [-1.0000e-02],
         [ 5.2080e+00],
         [-1.9775e+01]],

        [[ 5.4120e+00],
         [ 3.2760e+00],
         [-1.0895e+01],
         ...,
         [-1.0000e-02],
         [ 5.2080e+00],
         [-2.1240e+01]],

        [[ 5.9000e+00],
         [ 2.7870e+00],
         [-1.1383e+01],
         ...,
         [-1.0000e-02],
         [ 5.6970e+00],
         [-2.2217e+01]]],


       [[[-3.8660e+00],


In [4]:
labels = data['label']
labels.shape

(1080,)

In [5]:
# label 데이터 확장

labels = data['label']
labels.shape
repeated_arr = []
for i in range(len(labels)):
    repeated_arr.extend(np.full(64, labels[i]))

labels_1 = np.array(repeated_arr)
labels_1.shape

(69120,)

In [6]:
input_data.shape

(1080, 256, 64, 1)

In [7]:
input_data[0][:][:].squeeze().shape

(256, 64)

In [8]:
input_T = []
for i in range(1080):
    input_data_m = input_data[i][:][:].squeeze().T # 1080개 돌면서 squeeze -> (256, 64), 전치 -> (64, 256)
    input_T.append(input_data_m) # 빈 리스트에 추가

input_T = np.array(input_T)
input_T.shape

(1080, 64, 256)

In [9]:
# input data 스케일링

sclaer = StandardScaler()
input_data_0 = input_T.reshape(1080*64,256)
input_data_1 = sclaer.fit_transform(input_data_0)

In [10]:
input_data_1.shape

(69120, 256)

In [11]:
from sklearn.model_selection import train_test_split

# 데이터 준비

X = input_data_1
y = labels_1

# data split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

In [12]:
print("X_train 데이터 수", X_train.shape)
print("X_test 데이터 수", X_test.shape)

X_train 데이터 수 (55296, 256)
X_test 데이터 수 (13824, 256)


In [13]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

# 모델 생성
model = RandomForestClassifier(n_estimators=100, random_state=0)

# 모델 학습
model.fit(X_train, y_train)

# 모델 평가
y_pred = model.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)
y_pred


# 피처 중요도 결과 리스트에서 0.004 이상의 값과 해당 값의 인덱스를 출력
feature_importances = model.feature_importances_
important_features = [idx for idx, importance in enumerate(feature_importances) if importance >= 0.004]
input_data_1_selected = input_data_1[:, important_features]

# 결과 출력
print("새로운 데이터셋 모양:", input_data_1_selected.shape)

# # 결과 출력
# for idx, importance in important_features:
#     print("인덱스:", idx, "중요도:", importance)


Accuracy: 0.8459924768518519
새로운 데이터셋 모양: (69120, 90)


In [14]:
# 피쳐 삭제한 input data - split

X2 = input_data_1_selected
y2 = labels_1

X2_train, X2_test, y2_train, y2_test = train_test_split(X2, y2, test_size=0.2, random_state=0)

In [30]:
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.metrics import accuracy_score

# 그래디언트 부스팅 모델 초기화 및 설정
gb_model = GradientBoostingClassifier(n_estimators=300, learning_rate=0.03, max_depth=9, subsample=0.7, min_samples_leaf=3, random_state=0)

# 모델 학습
gb_model.fit(X2_train, y2_train)

# 테스트 데이터에 대한 예측
y2_pred = gb_model.predict(X2_test)
y2_train_hat = gb_model.predict(X2_train)

# 정확도 평가
accuracy = accuracy_score(y2_test, y2_pred)
train_accuracy = accuracy_score(y2_train, y2_train_hat)
print("train 데이터 정확도:", train_accuracy )
print("테스트 데이터 정확도:", accuracy)


train 데이터 정확도: 0.9634874131944444
테스트 데이터 정확도: 0.8158275462962963


In [16]:
# from sklearn.svm import SVC

# # RBF 커널을 사용하는 SVM 모델 훈련
# svm_model = SVC(kernel='rbf', C=1, gamma='scale', random_state=42)
# svm_model.fit(X2_train, y2_train)

# # 테스트 세트로 모델 평가
# y2_pred = svm_model.predict(X2_test)
# accuracy = accuracy_score(y2_test, y2_pred)
# print("Accuracy:", accuracy)
# y2_pred

## 제출용

In [17]:
# 진짜 테스트 데이터 로드

data_test=np.load('/content/drive/MyDrive/test.npy', allow_pickle='TRUE')
data_test = data_test.item()
data_input_test = data_test['input']


In [31]:

input_test_T = []
for i in range(120):
    data_input_m = data_input_test[i][:][:].squeeze().T
    input_test_T.append(data_input_m)

input_test_T = np.array(input_test_T)
input_test_T.shape


(120, 64, 256)

In [32]:
data_input_0_test = data_input_test.reshape(120*64,256)
data_input_1_test = sclaer.fit_transform(data_input_0_test)

In [33]:

data_input_1_test.shape

(7680, 256)

In [34]:
# 테스트 데이터 - 중요도 낮은 피쳐 제거

data_input_1_test_del = data_input_1_test[:, important_features]
data_input_1_test_del.shape

(7680, 90)

In [35]:

y_pred = gb_model.predict(data_input_1_test_del)

y_pred.mean()


0.6834635416666667

In [23]:
y_pred.shape

(7680,)

In [36]:

answer = []

for i in range(120):
    pred = []
    for s in range(64):
        pred.append(y_pred[i*64+s])
    if sum(pred)/len(pred) > 0.65: # 64개씩 그룹지어서 다수결 방식. 임계값 지정
        answer.append(1)
    else:
        answer.append(0)

len(answer)

120

In [37]:
sum(answer)/len(answer)

0.6

In [38]:
# 제출하기
id=np.arange(120)
target=answer

submission_df=pd.DataFrame({'ID':id, 'Target':target})

submission_df

,ID,Target
0,0,1
1,1,0
2,2,1
3,3,0
4,4,0
...,...,...
115,115,0
116,116,1
117,117,0
118,118,0


In [39]:
submission_df.to_csv('submission8.csv', index=False)